<a href="https://colab.research.google.com/github/d-m-silva/30DaysOfFLCode/blob/main/%22days/Day%2029%22/vanilla_encoding_decoding_ckks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CKKS explained: Part 1, Vanilla Encoding and Decoding

We will see in this notebook how to implement a Vanilla encoder and decoder in CKKS.

In [1]:
import numpy as np

# First we set the parameters
M = 8
N = M //2

# We set xi, which will be used in our computations
xi = np.exp(2 * np.pi * 1j / M)
xi

(0.7071067811865476+0.7071067811865475j)

In [2]:
from numpy.polynomial import Polynomial

class CKKSEncoder:
    """Basic CKKS encoder to encode complex vectors into polynomials."""

    def __init__(self, M: int):
        """Initialization of the encoder for M a power of 2.

        xi, which is an M-th root of unity will, be used as a basis for our computations.
        """
        self.xi = np.exp(2 * np.pi * 1j / M)
        self.M = M

    @staticmethod
    def vandermonde(xi: np.complex128, M: int) -> np.array:
        """Computes the Vandermonde matrix from a m-th root of unity."""

        N = M //2
        matrix = []
        # We will generate each row of the matrix
        for i in range(N):
            # For each row we select a different root
            root = xi ** (2 * i + 1)
            row = []

            # Then we store its powers
            for j in range(N):
                row.append(root ** j)
            matrix.append(row)
        return matrix

    def sigma_inverse(self, b: np.array) -> Polynomial:
        """Encodes the vector b in a polynomial using an M-th root of unity."""

        # First we create the Vandermonde matrix
        A = CKKSEncoder.vandermonde(self.xi, M)

        # Then we solve the system
        coeffs = np.linalg.solve(A, b)

        # Finally we output the polynomial
        p = Polynomial(coeffs)
        return p

    def sigma(self, p: Polynomial) -> np.array:
        """Decodes a polynomial by applying it to the M-th roots of unity."""

        outputs = []
        N = self.M //2

        # We simply apply the polynomial on the roots
        for i in range(N):
            root = self.xi ** (2 * i + 1)
            output = p(root)
            outputs.append(output)
        return np.array(outputs)

Now we can start experimenting with real values, let's first encode a vector and see how it is encoded.

In [3]:
# First we initialize our encoder
encoder = CKKSEncoder(M)

In [12]:
#b = np.array([1, 2, 3, 4])
b = np.array([1] + list(np.zeros(3)))
b

array([1., 0., 0., 0.])

Let's encode the vector now.

In [13]:
p = encoder.sigma_inverse(b)
p

Polynomial([ 2.50000000e-01-1.31838984e-16j,  1.76776695e-01-1.76776695e-01j,
        6.24500451e-17-2.50000000e-01j, -1.76776695e-01-1.76776695e-01j], domain=[-1,  1], window=[-1,  1], symbol='x')

Let's see now how we can extract the vector we had initially from the polynomial:

In [14]:
b_reconstructed = encoder.sigma(p)
b_reconstructed

array([ 1.00000000e+00+3.46944695e-17j, -5.55111512e-17+1.31838984e-16j,
       -1.66533454e-16+6.93889390e-18j, -1.11022302e-16-4.85722573e-17j])

We can see that the reconstruction and the initial vector are very close.

In [15]:
np.linalg.norm(b_reconstructed - b)

2.5324586304240974e-16

As stated before, $\sigma$ is not chosen randomly to encode and decode, but has a lot of nice properties. Among them, $\sigma$ is an isomorphism, therefore addition and multiplication on polynomials will result in coefficient wise addition and multiplication on the encoded vectors.

The homomorphic property of $\sigma$ is due to the fact that $X^N = 1$ and $\xi^N = 1$.

We can now start to encode several vectors, and see how we can perform homomorphic operations on them and decode it.

In [16]:
m1 = np.array([1, 2, 3, 4])
m2 = np.array([1, -2, 3, -4])

In [17]:
p1 = encoder.sigma_inverse(m1)
p2 = encoder.sigma_inverse(m2)

We can see that addition is pretty straightforward.

In [18]:
p_add = p1 + p2
p_add

Polynomial([ 2.00000000e+00+1.11022302e-16j, -7.07106781e-01+7.07106781e-01j,
        2.10942375e-15-2.00000000e+00j,  7.07106781e-01+7.07106781e-01j], domain=[-1.,  1.], window=[-1.,  1.], symbol='x')

Here as expected, we see that p1 + p2 decodes correctly to $[2, 0, 6, 0]$.

In [19]:
encoder.sigma(p_add)

array([2.0000000e+00+3.25176795e-17j, 4.4408921e-16-4.44089210e-16j,
       6.0000000e+00+1.11022302e-16j, 4.4408921e-16+3.33066907e-16j])

Because when doing multiplication we might have terms whose degree is higher than $N$, we will need to do a modulo operation using $X^N + 1$.

To perform multiplication, we first need to define the polynomial modulus which we will use.

In [20]:
poly_modulo = Polynomial([1,0,0,0,1])
poly_modulo

Polynomial([1., 0., 0., 0., 1.], domain=[-1,  1], window=[-1,  1], symbol='x')

Now we can perform multiplication.

In [21]:
p_mult = p1 * p2 % poly_modulo

Finally if we decode it, we can see that we have the expected result.

In [22]:
encoder.sigma(p_mult)

array([  1.-8.67361738e-16j,  -4.+6.86950496e-16j,   9.+6.86950496e-16j,
       -16.-9.08301212e-15j])